<a href="https://colab.research.google.com/github/IshikaGeed/HackOweek/blob/main/Hack_o_Week_Week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas matplotlib statsmodels streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.7 MB/s eta 0:00:00


In [17]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

import plotly.graph_objects as go


In [18]:
df = pd.read_csv("building_energy_data_extended.csv")


In [19]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)


In [20]:
df['Energy_Usage (kWh)'] = df['Energy_Usage (kWh)'].ffill()


In [21]:
# -------- FEATURE ENGINEERING --------

# Hour of day (0–23)
df['hour'] = df.index.hour

# Day of week (0=Monday, 6=Sunday)
df['day_of_week'] = df.index.dayofweek

# Rolling averages (trend capture)
df['rolling_3h'] = df['Energy_Usage (kWh)'].rolling(3).mean()
df['rolling_6h'] = df['Energy_Usage (kWh)'].rolling(6).mean()

# Fill NaNs created by rolling
df[['rolling_3h', 'rolling_6h']] = df[['rolling_3h', 'rolling_6h']].bfill()


In [22]:
X = df[['hour', 'day_of_week', 'rolling_3h', 'rolling_6h']]
y = df['Energy_Usage (kWh)']


In [23]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)


In [24]:
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [25]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

accuracy = 100 - (mae / y_test.mean()) * 100

print("Model Performance:")
print("MAE:", round(mae, 2))
print("R² Score:", round(r2, 3))
print("Accuracy (% approx):", round(accuracy, 2))


Model Performance:
MAE: 86.24
R² Score: 0.331
Accuracy (% approx): 69.03


In [26]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_test,
    name='Actual Usage'
))

fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_pred,
    name='Predicted Usage'
))

fig.update_layout(
    title="Library Energy Consumption During Exams",
    xaxis_title="Time",
    yaxis_title="Energy (kWh)"
)

fig.show()
